In [3]:
from collections import defaultdict, deque
from openai import OpenAI
from typing import Dict, Any, Optional
import datetime
import string
import random
import os
from dataclasses import dataclass

# Python decorator that automatically creates helpful methods in NPC Factory
@dataclass
class NPCConfig:
    world_description: str
    character_description: str
    history_size: int = 10
    has_scratchpad: bool = False

class NPCFactoryError(Exception):
    """Base exception class for NPC Factory errors."""
    pass

class NPCNotFoundError(NPCFactoryError):
    """Raised when trying to interact with a non-existent NPC."""
    def __init__(self, npc_id: str):
        self.npc_id = npc_id
        super().__init__(f"NPC with ID '{npc_id}' not found")

### class SimpleChatNPC purpose
- Stores the NPC’s configuration (personality, background, memory settings, scratchpad option)
- For user chatting with this NPC, it keeps user chat history
- Handles the back-and-forth chat between one user and this one NPC
- Knows how to use the LLM API to respond like its assigned character

In [4]:
class SimpleChatNPC:
    def __init__(self, client: OpenAI, model: str, config: NPCConfig):
        self.client = client
        self.model = model
        self.config = config
        self.chat_histories = defaultdict(lambda: deque(maxlen=config.history_size))

    def get_system_message(self) -> Dict[str, str]:
        """Returns the system message that defines the NPC's behavior."""
        character_description = self.config.character_description

        if self.config.has_scratchpad:
            character_description += """
You can use scratchpad for thinking before you answer: whatever you output in #SCRATCHPAD and #ANSWER won't be shown to anyone.
You start your output with #SCRATCHPAD and after you've done thinking, you #ANSWER"""

        return {
            "role": "system",
            "content": f"""WORLD SETTING: {self.config.world_description}
###
{character_description}"""
        }

    def chat(self, user_message: str, user_id: str) -> str:
        """Process a user message and return the NPC's response."""
        messages = [self.get_system_message()]

        # Add conversation history
        # Grab the previous conversation (history) between this user and the NPC.
        # If there is chat history, add it to the message list (so the LLM remembers what was said earlier).
        
        history = list(self.chat_histories[user_id])
        if history:
            messages.extend(history)

        # Add new user message
        # Make a dictionary for the new message (who said it and what they said).
        # Add it to the conversation history for this user.
        # Add it to the messages list to send to the LLM.
        
        user_message_dict = {
            "role": "user",
            "content": user_message
        }
        self.chat_histories[user_id].append(user_message_dict)
        messages.append(user_message_dict)

        # Ask LLM to generate a reply, using the chosen model, the conversation so far, and a set temperature.
        # Get the generated response (the NPC’s full reply, which may include a scratchpad depending if boolean is true or false).
        try:
            completion = self.client.chat.completions.create(
                model=self.model,
                messages=messages,
                temperature=0.6
            )

            response = completion.choices[0].message.content

            # Handle scratchpad if enabled
            # If scratchpads are enabled:
            # Look for the #SCRATCHPAD and #ANSWER markers in the response.
            # Extract only the part after #ANSWER (the actual answer for the user), and store it in response_clean variable.
            
            response_clean = response
            if self.config.has_scratchpad:
                import re
                scratchpad_match = re.search(r"#SCRATCHPAD(:?)(.*?)#ANSWER(:?)", response, re.DOTALL)
                if scratchpad_match:
                    response_clean = response[scratchpad_match.end():].strip()


            # Store response in history, including the scratchpad
            # Save the entire response (including scratchpad, if there is one) in the history for this user.
            
            self.chat_histories[user_id].append({
                "role": "assistant",
                "content": response
            })

            # Return the message to the user without a scratchpad
            return response_clean

        except Exception as e:
            return f"Error: {str(e)}"

### NPCFactory is the manager:

- It holds all registered NPCs.

- It keeps track of all users and their unique IDs.

- It has methods to Register users and NPCs, Handle chats between any user and any NPC, Retrieve the conversation history between a specific user and NPC

In [5]:
class NPCFactory:

    # When create an NPCFactory, have to give:
    # client: the connection to the LLM service.
    # model: the name of the language model to use.
    # It sets up two dictionaries:
        # self.npcs: for storing all the NPCs you make (key: NPC ID, value: NPC object).
        # self.user_ids: for storing all users (key: username, value: user’s unique ID).
    def __init__(self, client: OpenAI, model: str):
        self.client = client
        self.model = model
        self.npcs: Dict[str, SimpleChatNPC] = {}
        self.user_ids: Dict[str, str] = {}  # username -> user_id mapping

    # Makes a random 8-letter string (e.g., “aBcDeFgH”) to use as a unique ID for users or NPCs.
    def generate_id(self) -> str:
        """Generate a random unique identifier."""
        return ''.join(random.choice(string.ascii_letters) for _ in range(8))

    # Alows add/register a new user by name.
    # If the name is already taken, it keeps adding a number at the end until the name is unique (like "Alice", "Alice_1", "Alice_2", etc.).
    # Creates a new user ID (random string), stores it, and returns it.
    def register_user(self, username: str) -> str:
        """Register a new user and return their unique ID.
        If username already exists, appends a numerical suffix."""
        base_username = username
        suffix = 1

        # Keep trying with incremented suffixes until we find an unused name
        while username in self.user_ids:
            username = f"{base_username}_{suffix}"
            suffix += 1

        user_id = self.generate_id()
        self.user_ids[username] = user_id
        return user_id

    #Create a new NPC with the world description, character description, memory size, and scratchpad option.
    # Generates a unique ID for this NPC.
    def register_npc(self, world_description: str, character_description: str,
                     history_size: int = 10, has_scratchpad: bool = False) -> str:
        """Create and register a new NPC, returning its unique ID."""
        npc_id = self.generate_id()
        
        # Creates a configuration object (NPCConfig) to neatly store all the NPC’s settings.
        config = NPCConfig(
            world_description=world_description,
            character_description=character_description,
            history_size=history_size,
            has_scratchpad=has_scratchpad
        )
        
        # Creates the actual NPC object (a chatbot) using all the details above.
        # Stores it in the npcs dictionary using its unique ID.
        # Returns the NPC’s ID so you can use it for chatting.
        
        self.npcs[npc_id] = SimpleChatNPC(self.client, self.model, config)
        return npc_id

    # Handles chatting! Code give it the NPC ID, user ID, and message.
    # If the NPC exists, it sends the message to that NPC, and returns the NPC’s reply.
    # If not, it throws an error.
    
    def chat_with_npc(self, npc_id: str, user_id: str, message: str) -> str:
        """Send a message to a specific NPC from a specific user.

        Args:
            npc_id: The unique identifier of the NPC
            user_id: The unique identifier of the user
            message: The message to send

        Returns:
            The NPC's response

        Raises:
            NPCNotFoundError: If the specified NPC doesn't exist
        """
        if npc_id not in self.npcs:
            raise NPCNotFoundError(npc_id)

        npc = self.npcs[npc_id]
        return npc.chat(message, user_id)

    # Go fetch the full chat history (list of all messages) between a specific user and NPC.
    # Returns the list (in order) so can see the whole conversation.
    
    def get_npc_chat_history(self, npc_id: str, user_id: str) -> list:
        """Retrieve chat history between a specific user and NPC.

        Args:
            npc_id: The unique identifier of the NPC
            user_id: The unique identifier of the user

        Returns:
            List of message dictionaries containing the chat history

        Raises:
            NPCNotFoundError: If the specified NPC doesn't exist
        """
        if npc_id not in self.npcs:
            raise NPCNotFoundError(npc_id)

        return list(self.npcs[npc_id].chat_histories[user_id])

### Connect to Nebius LLM service

In [6]:
with open("nebius_api_key", "r") as file:
    nebius_api_key = file.read().strip()

os.environ["NEBIUS_API_KEY"] = nebius_api_key

from openai import OpenAI

# Nebius uses the same OpenAI() class, but with additional details
client = OpenAI(
    base_url="https://api.studio.nebius.ai/v1/",
    api_key=os.environ.get("NEBIUS_API_KEY"),
)

model = "meta-llama/Meta-Llama-3.1-405B-Instruct"

# Creating a factory
npc_factory = NPCFactory(client=client, model=model)

### To format the LLM output to be more readable

In [7]:
def prettify_string(text, max_line_length=80):
    """Prints a string with line breaks at spaces to prevent horizontal scrolling.

    Args:
        text: The string to print.
        max_line_length: The maximum length of each line.
    """

    output_lines = []
    lines = text.split("\n")
    for line in lines:
        current_line = ""
        words = line.split()
        for word in words:
            if len(current_line) + len(word) + 1 <= max_line_length:
                current_line += word + " "
            else:
                output_lines.append(current_line.strip())
                current_line = word + " "
        output_lines.append(current_line.strip())  # Append the last line
    return "\n".join(output_lines)

### Create user and a NPC. With ScratchPad

In [8]:
# Register and create a user
user_id = npc_factory.register_user("ChongMing")

# Create an NPC
npc_id = npc_factory.register_npc(
    world_description="""Modern financial centers—from Singapore to London—are powered by sophisticated transaction banking networks. 
                         Payments, collections, and liquidity management are no longer manual; 
                         AI-driven systems and cloud platforms automate most treasury tasks. 
                         Corporates expect instant, transparent cross-border payments and intelligent cash pooling. 
                         The role of transaction banking is more critical than ever to support global supply chains, manage risk, 
                         and fuel growth in a volatile economic landscape """,
    
    character_description="""You are a Senior Product Manager for Digital Payments at international bank. 
                            You are innovative, customer-focused, and love to solve problems for large corporates. 
                            You oversee the design and rollout of real-time payment solutions, collections platforms, and API-based treasury products. 
                            Your goal is to help clients move money faster, reconcile payments easily, and integrate banking with their internal systems. 
                            You speak in a friendly, consultative tone, sometimes using analogies from technology or sports. 
                            You ask about client pain points and propose tailored solutions. 
                            You also keep an eye on industry trends and enjoy discussing the future of payments.""",
    has_scratchpad=True
)

In [9]:
response = npc_factory.chat_with_npc(npc_id, user_id,
                                     """Hi, I want to understand the difference between real time gross settlement and net settlement systems.
                                     Can you help me to list out 3 unique features. That's for my thesis."""
                                     )
print(prettify_string(response))

Hi, I'd be happy to help you with your thesis. Here are 3 unique features each
of real-time gross settlement (RTGS) and net settlement systems:

**Real-Time Gross Settlement (RTGS) Features:**

1. **Real-time processing**: Each transaction is settled individually and
immediately, ensuring fast and efficient payment processing.
2. **Finality of payment**: Once a payment is made, it cannot be revoked,
providing a high level of certainty and reducing the risk of payment reversals.
3. **Liquidity requirements**: Banks need to have sufficient liquidity to
settle each transaction in real-time, which can be a challenge, especially
during peak periods.

**Net Settlement Features:**

1. **Batch processing**: Transactions are settled in batches, usually at the
end of the day, which can help reduce the processing load and minimize errors.
2. **Netting**: All transactions between two banks are netted off against each
other, reducing the total amount that needs to be settled and minimizing the
risk

In [10]:
npc_factory.get_npc_chat_history(npc_id, user_id)

[{'role': 'user',
  'content': "Hi, I want to understand the difference between real time gross settlement and net settlement systems.\n                                     Can you help me to list out 3 unique features. That's for my thesis."},
 {'role': 'assistant',
  'content': "#SCRATCHPAD \nReal-time gross settlement (RTGS) and net settlement systems are two different approaches used for settling transactions between banks. Here are the definitions and differences to help me answer the question: \n\n- RTGS systems settle transactions individually and in real-time, meaning that the payment is processed and settled immediately. This approach ensures that the payment is final and irrevocable as soon as it is made.\n\n- Net settlement systems, on the other hand, settle transactions in batches, typically at the end of the day. In this approach, all the transactions between two banks are netted off against each other, and only the net amount is settled.\n\nNow, let's extract 3 unique fea